In [1]:
from my_functions import create_corpus_file, process_dataset, reddit_data_cleaning, train_word2vec, load_thesis_datasets,\
generate_word2vector, generate_word_embeddings, setup_rnn, generate_rnn_model, dataset_shuffler

import time
import logging
import pandas as pd
import numpy as np
import nltk
import re
import os

import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding, LeakyReLU
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.models import Model
from sklearn.model_selection import train_test_split

In [2]:
#First we Load the word2vec models we have trained

wikicorpus = 'glove.6B.200d.txt'

reddit_cbow_dim100_win5 = 'reddit_model_CBOW_dim100_con5.txt'
reddit_cbow_dim100_win10 = 'reddit_model_CBOW_dim200_con10.txt'
reddit_cbow_dim200_win5 = 'reddit_CBOW_DIM200_win5.txt'
reddit_cbow_dim200_win10 = 'reddit_model_CBOW_dim200_con10.txt'
reddit_bow_dim300_win10 = 'reddit_model_CBOW_dim300_con10.txt'
reddit_sg_dim200_win3 = 'reddit_model_SG_dim200_win3.txt'
reddit_sg_dim200_win5 = 'reddit_model_SG_dim200_win5.txt'
reddit_sg_dim200_win10 = 'reddit_sg_dim300_win5.txt'
reddit_sg_dim300_win5 = 'reddit_sg_dim300_win5.txt'
reddit_sg_dim300_win10 = 'reddit_model_SG_dim300_win10.txt'

transcript_sg_dim100_win5 = 'transcript_SG_DIM100_win5.txt'
transcript_sg_dim100_win10 = 'transcript_SG_DIM100_win10.txt'
transcript_sg_dim200_win5 = 'transcript_SG_DIM200_win5.txt'
transcript_sg_dim200_win10 = 'transcript_SG_DIM200_win10.txt'
transcript_sg_dim300_win5 = 'transcript_SG_DIM300_WIN5.txt'
transcript_sg_dim300_win10 = 'transcript_sg_DIM300_win10-new.txt'
transcript_cbow_dim100_win5 = 'transcript_CBOW_DIM100_win5.txt'
transcript_cbow_dim100_win10 = 'transcript_CBOW_DIM100_win10.txt'
transcript_cbow_dim200_win5 = 'transcript_CBOW_DIM200_win5.txt'
transcript_cbow_dim200_win10 = 'transcript_CBOW_DIM200_win10.txt'
transcript_cbow_dim300_win5 = 'transcript_CBOW_DIM300_win5.txt'
transcript_cbow_dim300_win10 = 'transcript_CBOW_DIM300_win10.txt'


In [3]:
# Load models and retrieve word vectors
# Un-comment the word-vectors you wish to use

# REDDIT SKIP-GRAM MODELS

# reddit_sg_dim100_win10_model = KeyedVectors.load(reddit_sg_dim100_win10)
# reddit_sg_dim100_win10_vectors = reddit_sg_dim100_win10_model.wv

# reddit_sg_dim200_win5_model = KeyedVectors.load(reddit_sg_dim200_win5)
# reddit_sg_dim200_win5_vectors = reddit_sg_dim200_win5_model.wv

# reddit_sg_dim200_win10_model = KeyedVectors.load(reddit_sg_dim200_win10)
# reddit_sg_dim200_win10_vectors = reddit_sg_dim200_win10_model.wv

# reddit_sg_dim300_win5_model = KeyedVectors.load(reddit_sg_dim300_win5)
# reddit_sg_dim300_win5_vectors = reddit_sg_dim300_win5_model.wv

# reddit_sg_dim300_win10_model = KeyedVectors.load(reddit_sg_dim300_win10) #NOT WORKING
# reddit_sg_dim300_win10_vectors = reddit_sg_dim300_win10_model.wv

# reddit_sg_dim100_win10_model = KeyedVectors.load(reddit_sg_dim300_win5)
# reddit_sg_dim100_win10_vectors = reddit_sg_dim300_win10_model.wv

# REDDIT CBOW MODELS

# reddit_cbow_dim100_win5_model = KeyedVectors.load(reddit_cbow_dim100_win5)
# reddit_cbow_dim100_win5_vectors = reddit_cbow_dim100_win5_model.wv

# reddit_cbow_dim100_win10_model = KeyedVectors.load(reddit_cbow_dim100_win10)
# reddit_cbow_dim100_win10_vectors = reddit_cbow_dim100_win10_model.wv

# reddit_cbow_dim200_win5_model = KeyedVectors.load(reddit_cbow_dim200_win5)
# reddit_cbow_dim200_win5_vectors = reddit_cbow_dim200_win5_model.wv

# reddit_cbow_dim200_win10_model = KeyedVectors.load(reddit_cbow_dim200_win10)
# reddit_cbow_dim200_win10_vectors = reddit_cbow_dim200_win10_model.wv

# reddit_bow_dim300_win10_model = KeyedVectors.load(reddit_bow_dim300_win10)
# reddit_bow_dim300_win10_vectors = reddit_bow_dim300_win10_model.wv

# TRANSCRIPT SKIP-GRAM MODELS

# transcript_sg_dim100_win5_model = KeyedVectors.load(transcript_sg_dim100_win5)
# transcript_sg_dim100_win5_vectors = transcript_sg_dim100_win5_model.wv

# transcript_sg_dim100_win10_model = KeyedVectors.load(transcript_sg_dim100_win10)
# transcript_sg_dim100_win10_vectors = transcript_sg_dim100_win10_model.wv

# transcript_sg_dim200_win5_model = KeyedVectors.load(transcript_sg_dim200_win5)
# transcript_sg_dim200_win5_vectors = transcript_sg_dim200_win5_model.wv

# transcript_sg_dim200_win10_model = KeyedVectors.load(transcript_sg_dim200_win10)
# transcript_sg_dim200_win10_vectors = transcript_sg_dim200_win10_model.wv

# transcript_sg_dim300_win5_model = KeyedVectors.load(transcript_sg_dim300_win5)
# transcript_sg_dim300_win5_vectors = transcript_sg_dim300_win5_model.wv

# transcript_sg_dim300_win10_model = KeyedVectors.load(transcript_sg_dim300_win10)
# transcript_sg_dim300_win10_vectors = transcript_sg_dim300_win10_model.wv

# TRANSCRIPT CBOW MODELS

# transcript_cbow_dim100_win5_model = KeyedVectors.load(transcript_cbow_dim100_win5)
# transcript_cbow_dim100_win5_vectors = transcript_cbow_dim100_win5_model.wv

# transcript_cbow_dim100_win10_model = KeyedVectors.load(transcript_cbow_dim100_win10)
# transcript_cbow_dim100_win10_vectors = transcript_cbow_dim100_win10_model.wv

# transcript_cbow_dim200_win5_model = KeyedVectors.load(transcript_cbow_dim200_win5)
# transcript_cbow_dim200_win5_vectors = transcript_cbow_dim200_win5_model.wv

# transcript_cbow_dim200_win10_model = KeyedVectors.load(transcript_cbow_dim200_win10)
# transcript_cbow_dim200_win10_vectors = transcript_cbow_dim200_win10_model.wv

# transcript_cbow_dim300_win5_model = KeyedVectors.load(transcript_cbow_dim300_win5)
# transcript_cbow_dim300_win5_vectors = transcript_cbow_dim300_win5_model.wv

transcript_cbow_dim300_win10_model = KeyedVectors.load(transcript_cbow_dim300_win10)
transcript_cbow_dim300_win10_vectors = transcript_cbow_dim300_win10_model.wv





In [ ]:
# Load WikiCorpus vectors
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(wikicorpus, tmp_file)
wikicorpus_model = KeyedVectors.load_word2vec_format(tmp_file)

In [4]:
df = process_dataset(load_thesis_datasets(), '2', '2-cleaned')

/home/opc/my_functions.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  data[cleaned_col] = data[cleaned_col].str.replace(pat, '')
/home/opc/my_functions.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  data[cleaned_col] = data[cleaned_col].str.replace(r'\s+', ' ')


Processing completed in: 6 seconds.


In [5]:
MAX_SEQUENCE_LENGTH = 21000
EMBEDDING_DIM = 300

In [6]:
# Splitting our dataset

data_train, data_test = train_test_split(df, test_size=0.20, random_state=42)

In [7]:
# Build our training vocabulary
training_words = [word for tokens in data_train['tokens'] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train['tokens']]
TRAINING_VOCAB = sorted(list(set(training_words)))
print('%s words total, with a vocabulary size of %s' % (len(training_words), len(TRAINING_VOCAB)))
print('Max sentence length is %s' % max(training_sentence_lengths))


# Build our test vocabulary
test_words = [word for tokens in data_test['tokens'] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test['tokens']]
TEST_VOCAB = sorted(list(set(test_words)))
print('%s words total, with a vocabulary size of %s' % (len(test_words), len(TEST_VOCAB)))
print('Max sentence length is %s' % max(test_sentence_lengths))

# PASS THE VECTORS FROM WORD2VEC HERE TO GENERATE TRAINING EMBEDDINGS
training_embeddings = generate_word_embeddings(transcript_cbow_dim300_win10_vectors, data_train, generate_missing=True)
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=True)
tokenizer.fit_on_texts(data_train["tokens"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["tokens"].tolist())
train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_rnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))

for word,index in train_word_index.items():
    train_embedding_weights[index,:] = transcript_cbow_dim300_win10_vectors[word] if word in transcript_cbow_dim300_win10_vectors\
    else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

test_sequences = tokenizer.texts_to_sequences(data_test["2-cleaned"].tolist())
test_rnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

label_names = ['Consp', 'NonConsp']
y_train = data_train[label_names].values
y_test = data_train[label_names].values
x_train = train_rnn_data

877230 words total, with a vocabulary size of 40012
Max sentence length is 20148
184376 words total, with a vocabulary size of 18219
Max sentence length is 16679
Found 40012 unique tokens.
(40013, 300)


In [8]:
# Initiate our RNN Classifier
model = setup_rnn(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21000)]           0         
_________________________________________________________________
embedding (Embedding)        (None, 21000, 300)        12003900  
_________________________________________________________________
lstm (LSTM)                  (None, 64)                93440     
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 12,105,918
Trainable params: 102,018
Non-trainable params: 12,003,900
___________________________________________

In [10]:
# Specify epochs and batch-size
num_epochs = 10
batch_size = 32

In [ ]:
# Train the RNN
hist = model.fit(x_train, y_train, epochs=num_epochs, validation_split=0.2, shuffle=True, batch_size=batch_size)


Epoch 1/10
 2/12 [====>.........................] - ETA: 5:25 - loss: 0.6968 - acc: 0.4922 

In [ ]:
# How to save model weights after training
model_json = model.to_json()
with open("modelname_weights.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("modelname_weights.h5")

In [ ]:
# # CODE FOR SAVING THE RESULTS OF OUR RNN
# # serialize model to JSON
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")
 
# # later...
 
# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")